# Dlib Demo

> Refer to: https://github.com/ageitgey/face_recognition/blob/master/face_recognition/api.py

> Refer to: http://dlib.net, http://dlib.net/files/

## Prepare

### Define dlib model variables

In [ ]:
from PIL import Image, ImageFile, ImageDraw

import dlib
import numpy as np
import matplotlib.pyplot as plt

dlib_face_recognition_resnet_model_v1 = 'dlib_face_recognition_resnet_model_v1.dat'
mmod_human_face_detector = 'mmod_human_face_detector.dat'
shape_predictor_5_face_landmarks = 'shape_predictor_5_face_landmarks.dat'
shape_predictor_68_face_landmarks = 'shape_predictor_68_face_landmarks.dat'

ImageFile.LOAD_TRUNCATED_IMAGES = True

### Make dataset

In [ ]:
from os import curdir, path
from itertools import chain

import glob
import random as rdm


class Dataset(object):
    def __init__(self):
        pic_dir = path.abspath(path.join(curdir, 'pics'))
        pic_files = set(glob.glob(path.join(picture_dir, '**/*.jpg')))
        self._dataset = self._make_dataset(pic_files)

    @staticmethod
    def _make_dataset(pic_files):
        dataset = {}
        for pf in pic_files:
            key = path.split(path.dirname(pf))[1]
            if key in dataset:
                dataset[key].append(pf)
            else:
                dataset[key] = [pf]
        return dataset

    def one_face(self):
        key = rdm.choice(list(self._dataset.keys()))
        return key, rdm.choice(self._dataset[key])

    def fetch(self, split_count=1):
        train_labels, train_data, test_labels, test_data = [], [], [], []
        for name, files in self._dataset.items():
            test_indices = np.random.choice(len(files), split_count)
            data_indices = np.setdiff1d(np.arange(len(files)), test_indices)

            files = np.array(files)
            train_labels.append(np.repeat(name, len(data_indices)))
            train_data.append(files[data_indices])

            test_labels.append(np.repeat(name, len(test_indices)))
            test_data.append(files[test_indices])

        return np.array(train_labels), np.array(train_data), np.array(test_labels), np.array(test_data)


dataset = Dataset()

### Utils functions

In [ ]:
def draw_rectange(img, rect, outline='#fff'):
    for n in range(0, 5):
        box = [(rect[0][0] + n, rect[0][1] + n), (rect[1][0] - n, rect[1][1] - n)]
        draw.rectangle(box, outline=outline)

## Create dlib model download function

In [169]:
import requests
import bz2
from os import curdir, path, makedirs, rename
from urllib.parse import urljoin

def download_model(model_file):
    def make_model_path(model_file, middle_path=''):
        base_dir = path.abspath(path.join(curdir, 'models', middle_path))
        if not path.exists(base_dir):
            makedirs(base_dir, exist_ok=True)

        return path.join(base_dir, model_file)

    local_model_file = make_model_path(model_file)
    if path.exists(local_model_file):
        print('* model file "{}" exsits'.format(model_file))
        return local_model_file

    print('* model file "{}" not exsits'.format(model_file))

    def unzip_bz2(src_file, dest_file):
        with bz2.BZ2File(src_file, 'rb') as src_fp, open(dest_file, 'wb') as dest_fp:
            while 1:
                data = src_fp.read(100 * (1024 ** 2))
                if not data:
                    break
                dest_fp.write(data)

    bz2_model_file = '{}.bz2'.format(model_file)
    bz2_local_model_file = make_model_path(bz2_model_file, middle_path='.cache')
    if path.exists(bz2_local_model_file):
        print('* model bz2 file "{}" exsits, uncompress it...'.format(bz2_local_model_file))
        unzip_bz2(bz2_local_model_file, local_model_file)

        print('* model bz2 file "{}" uncompressed, save as "{}"'.format(bz2_local_model_file, local_model_file))
        return local_model_file

    download_url = urljoin('http://dlib.net/files/', bz2_model_file)
    print('* begin download from url "{}"'.format(download_url))

    r = requests.get(download_url, stream=True, verify=False)
    total_size = int(r.headers.get('Content-Length', -1))
    print('* total file size is {}'.format(total_size))

    download_size = 0
    download_tmp_file = '{}__download.tmp'.format(bz2_local_model_file)
    with open(download_tmp_file, 'wb') as fp:
        print('* create download file "{}"'.format(download_tmp_file))
        for chunk in r.iter_content(chunk_size=10 * (1024 ** 2)):
            if chunk:
                fp.write(chunk)
                download_size += len(chunk)
                print('* {:.2f}kb downloaded'.format(download_size / 1024), end='\r')
            else:
                print('* download completed')

    rename(download_tmp_file, bz2_local_model_file)
    print('* rename "{}" to "{}"'.format(download_tmp_file, bz2_local_model_file))

    unzip_bz2(bz2_local_model_file, local_model_file)
    print('* model bz2 file "{}" uncompressed, save as "{}"'.format(bz2_local_model_file, local_model_file))
    
    return local_model_file

## Face detect

- Load face image

In [ ]:
_, face_file = dataset.one_face()
detect_im = np.array(Image.open(face_file).convert('RGB'))

plt.imshow(detect_im)
plt.show()

### HOG algorithm

In [ ]:
face_detector = dlib.get_frontal_face_detector()

boxes = face_detector(detect_im, 1)
print('* {} faces detected, boxes is "{}"'.format(len(boxes), boxes))

img = Image.fromarray(detect_im)
draw = ImageDraw.Draw(img)

for box in boxes:
    box = [(box.left(), box.top()), (box.right(), box.bottom())]
    draw_rectange(draw, box, outline='#fff')

plt.imshow(img)
plt.show()

### CNN algorithm

In [ ]:
model_file = download_model(mmod_human_face_detector)

if not cnn_face_detector:
    cnn_face_detector = dlib.cnn_face_detection_model_v1(model_file)

boxes = cnn_face_detector(detect_im, 1)
print('* {} faces detected, boxes is "{}"'.format(len(boxes), boxes))

img = Image.fromarray(detect_im)
draw = ImageDraw.Draw(img)

for box in boxes:
    box = box.rect
    box = [(box.left(), box.top()), (box.right(), box.bottom())]
    draw_rectange(draw, box, outline='#fff')

plt.imshow(img)
plt.show()